<a href="https://colab.research.google.com/github/PisutSukpool/BADS/blob/main/Social%20Network%20Analysis/Cascading_Twitch_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prep Data

In [1]:
#After Processing Community Detection and PageRank
import networkx as nx
import pandas as pd
import copy
import itertools
#Create Datafram of Nodes and Edges
df_node = pd.read_csv('https://github.com/PisutSukpool/BADS/raw/main/Social%20Network%20Analysis/Node_Community_2%2BPageRank.csv')  
df_edge = pd.read_csv('https://github.com/PisutSukpool/BADS/raw/main/Social%20Network%20Analysis/Edge_Community_2.csv')  
df_node_cl = df_node[['Id','Label','Interest','pageranks','clustering','betweenesscentrality']]

In [2]:
#Create Edge , Nodes and Reset Data
G = nx.from_pandas_edgelist(df_edge,source='Source',
                                   target='Target',edge_attr=True,
                                   create_using=nx.Graph())
print('Edges:',G.number_of_edges(),'  Nodes:',G.number_of_nodes())

#Iterate over df rows and set the source and target nodes' attributes for each row:
for index, row in df_node_cl.iterrows():
    G.nodes[row['Id']]['Label'] = row['Label']
    G.nodes[row['Id']]['Interest'] = row['Interest']
    G.nodes[row['Id']]['pageranks'] = round(row['pageranks'],10)
    G.nodes[row['Id']]['clustering'] = round(row['clustering'],10)
    G.nodes[row['Id']]['betweenesscentrality'] = round(row['betweenesscentrality'],10)
    
def set_and_refresh():
    global G
    global df_edge
    global df_node_cl
    G = nx.from_pandas_edgelist(df_edge,source='Source',
                                       target='Target',edge_attr=True,
                                       create_using=nx.Graph())
    #print('Edges:',G.number_of_edges(),'  Nodes:',G.number_of_nodes())

    #Iterate over df rows and set the source and target nodes' attributes for each row:
    for index, row in df_node_cl.iterrows():
        G.nodes[row['Id']]['Label'] = row['Label']
        G.nodes[row['Id']]['Interest'] = row['Interest']
        G.nodes[row['Id']]['pageranks'] = round(row['pageranks'],10)
        G.nodes[row['Id']]['clustering'] = round(row['clustering'],10)
        G.nodes[row['Id']]['betweenesscentrality'] = round(row['betweenesscentrality'],10)
    #print('<<<Complete Refresh Network>>')    

Edges: 12778   Nodes: 6958


In [21]:
#Export Network Data to visual in Gephi
def export_to_csv(csv_name):
    global G
    data={}
    data['Id']=[x[0] for x in G.nodes(data=True)]
    data['Label'] = [x[1]['Label'] for x in G.nodes(data=True)]
    data['Interest'] = [x[1]['Interest'] for x in G.nodes(data=True)]
    data['pageranks'] = [x[1]['pageranks'] for x in G.nodes(data=True)]
    data['clustering'] = [x[1]['clustering'] for x in G.nodes(data=True)]
    data['betweenesscentrality'] = [x[1]['betweenesscentrality'] for x in G.nodes(data=True)]
    df_node_export = pd.DataFrame(data)
    df_node_export.to_csv(csv_name+'.csv',index=False)
    print('<<Complete Export Network>>')

In [15]:
#Export set of adopted node to visual in Excel
def export_set_adopted_node(time,csv_name):
    if time == 'one':
        df_adopted_onetime_stg.to_csv(csv_name+'.csv', index=False)
    elif time == 'converge':
        df_adopted_converge_stg.to_csv(csv_name+'.csv', index=False)
    print('Finish export_set_adopted_node')

In [24]:
#Select Top 5 PageRanks and Create Superset of that users
num_top = int(input('Enter Number of Top PageRanks : '))
user=[row['Id'] for index,row in df_node_cl.iterrows() if row['Label'] < num_top+1]
print('User Top 5 PageRanks :',user)
lst_set=[list(itertools.combinations(user, i)) for i in range(1, len(user)+1)]  #Create Superset of user
print('Superset of Top 5 PageRanks :',lst_set)

Enter Number of Top PageRanks : 2
User Top 5 PageRanks : ['alicetvz', 'znebtv']
Superset of Top 5 PageRanks : [[('alicetvz',), ('znebtv',)], [('alicetvz', 'znebtv')]]


Start Cascading

In [17]:
#Initial Viral Top 5 Nodes by PageRanks
def begin_viral(game_adopted):
    global G
    for v in user:
        G.nodes(data=True)[v]['Interest'] = game_adopted
        print('     User :',v,' Adopted')

In [18]:
#Cascading One Time
def viral_onetime(seed,original,game_adopted):
        global G
        global df_adopted_onetime_stg
        jst_adopt = list(seed)
        adopted = copy.deepcopy(jst_adopt)                                                  
        tmp=[]
        for v in jst_adopt: #node is seed
            G.nodes(data=True)[v]['Interest'] = game_adopted #change game
            for nb in G.neighbors(v): # neighbors of node
                nb_adopted=0
                nb_noadopted=0
                if len([n for n in G.neighbors(nb) if n != v]) > 0 : # check neighbor has other neighbors
                    for nb2 in G.neighbors(nb):  #neighbors of neighbor   
                        if G.nodes(data=True)[nb]['Interest'] == game_adopted: # ถ้า nb adopted อยู่แล้วไม่ต้องลูปต่อ
                            nb_adopted+=1
                        else:
                            for attr in G.nodes(data=True)[nb2]: #attribute neighbors
                                if attr=='Interest':
                                    if G.nodes(data=True)[nb2][attr] == original:
                                        nb_noadopted+=1
                                    else:
                                        nb_adopted+=1  
                else:
                    nb_adopted+=1 #nb no neighbors
                if nb_adopted+nb_noadopted != 0:
                    threshold = nb_noadopted/(nb_adopted+nb_noadopted)
                    q=threshold
                    p=1-q
                    if p>q and nb not in adopted and nb not in tmp:
                        tmp.append(nb)
                        G.nodes(data=True)[nb]['Interest'] = game_adopted #Adopted!!!!!!  str(
        for v in tmp:
            adopted.append(v)
        df_adopted_onetime_stg = pd.concat([df_adopted_onetime_stg,pd.DataFrame([{'set_of_nodes':seed
                                                                            ,'number_of_iteration':1
                                                                            ,'number_of_adopted_nodes':len(adopted)}])],axis=0)
        return len(adopted)

In [19]:
#Cascading Until Converge
    #G --> Network
    #seed --> List of Node Begin Viral
    #original --> Old genre of game
    #game_adopted --> Game which we want to viral
def viral_converge(seed,original,game_adopted):
        global G
        global df_adopted_converge_stg    
        #jst_adopt --> List of Nodes that just adopt
        #adopted --> List of All nodes that adopt
        #num_iter_adopted --> Number of Iteration
        jst_adopt = list(seed)
        adopted = copy.deepcopy(jst_adopt)
        num_iter_adopted = 0 
        while(1):
            if len(jst_adopt)==0:
                return num_iter_adopted, len(adopted) ,df_adopted_converge_stg
            tmp=[]
            for v in jst_adopt: #Initial Node
                G.nodes(data=True)[v]['Interest'] = game_adopted #Node begin viral
                for nb in G.neighbors(v): # Neighbors of Nodes
                    nb_adopted=0
                    nb_noadopted=0
                    if len([n for n in G.neighbors(nb) if n != v]) > 0 : # check neighbor has other neighbors
                        for nb2 in G.neighbors(nb):  #neighbors of neighbor   
                            if G.nodes(data=True)[nb]['Interest'] == game_adopted: # if neighbors of nb adopted 
                                nb_adopted+=1
                            else:
                                for attr in G.nodes(data=True)[nb2]: #attribute neighbors
                                    if attr=='Interest':
                                        if G.nodes(data=True)[nb2][attr] == original:
                                            nb_noadopted+=1
                                        else:
                                            nb_adopted+=1  
                    else:
                        nb_adopted+=1 
                    if nb_adopted+nb_noadopted != 0:
                        threshold = nb_noadopted/(nb_adopted+nb_noadopted)  #Set Payoff Threshold
                        q=threshold
                        p=1-q
                        if p>q and nb not in adopted and nb not in tmp: # Adopted Condition
                            tmp.append(nb)
                            G.nodes(data=True)[nb]['Interest'] = game_adopted #Adopted!!!!!!
            for v in tmp:
                adopted.append(v)
            jst_adopt=tmp #Bring New Adopted Node to spread again
            if len(jst_adopt) != 0 :
                num_iter_adopted+=1
                df_adopted_converge_stg = pd.concat([df_adopted_converge_stg,pd.DataFrame([{'set_of_nodes':seed
                                                                            ,'number_of_iteration':num_iter_adopted
                                                                            ,'number_of_adopted_nodes':len(adopted)}])])
                print('seed: ',seed,'iter:',num_iter_adopted,' cum_adopted: ',len(adopted))

In [25]:
#Execute Cascading
game_adopted = str(input('Enter the game which want to viral : ')) #'Among Us'
how_spread = int(input('Enter How Spread (Begin Viral : 1 | Spread One : 2 | Spread Converge : 3): ')) # 1,2,3

if how_spread == 1:    
    begin_viral(game_adopted)
    export_to_csv('Gephi_Begin_Viral_Nodes') 
    set_and_refresh()
    
elif how_spread == 2:
    df_adopted_onetime_stg = pd.DataFrame()
    for major in lst_set:
        for minor in major:
            viral_onetime(minor,'Survival',game_adopted)
            set_and_refresh()
    export_to_csv('Gephi_Spread_Onetime_Nodes')
    export_set_adopted_node('one','One_Set_Adopted_Nodes')
    set_and_refresh()
    
elif how_spread == 3:
    df_adopted_converge_stg = pd.DataFrame()
    for major in lst_set:
        for minor in major:    
            viral_converge(minor,'Survival',game_adopted) 
            set_and_refresh() 
    export_to_csv('Gephi_Spread_Converge_Nodes') 
    export_set_adopted_node('converge','Converge_Set_Adopted_Nodes')
    set_and_refresh()    
    
else:
    print('Input type of how_spread agin!')

Enter the game which want to viral : Among Us
Enter How Spread (Begin Viral : 1 | Spread One : 2 | Spread Converge : 3): 3
seed:  ('alicetvz',) iter: 1  cum_adopted:  1919
seed:  ('znebtv',) iter: 1  cum_adopted:  558
seed:  ('alicetvz', 'znebtv') iter: 1  cum_adopted:  3217
seed:  ('alicetvz', 'znebtv') iter: 2  cum_adopted:  3220
<<Complete Export Network>>
Finish export_set_adopted_node
